<a href="https://colab.research.google.com/github/H0wAreU/Statistical-Learning-Labs./blob/main/Lab03_kNN_using_Grid_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Statistical Learning for Data Science 2 (229352)
#### Instructor: Donlapark Ponnoprat

#### [Course website](https://donlapark.pages.dev/229352/)

## Lab #3

In [ ]:
from sklearn.datasets import fetch_20newsgroups

train = fetch_20newsgroups(subset='train')
test = fetch_20newsgroups(subset='test')

Xtrain = train.data[:3000]
ytrain = train.target[:3000]
Xtest = test.data[:500]
ytest = test.target[:500]

print("X:", len(Xtest))
print("y:", len(ytest))

X: 500
y: 500


In [ ]:
print("X[0]:", Xtrain[0])
print("y[0]:", ytrain[0])

X[0]: From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

Thanks,
- IL
   ---- brought to you by your neighborhood Lerxst ----





y[0]: 7


In [ ]:
train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

### Apply Tfidf ([documentation](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# define transformation
tfidf = TfidfVectorizer(stop_words='english',ngram_range=(1,1))

# fit+transform training set
Xtrain_tfidf = tfidf.fit_transform(Xtrain)
Xtest_tfidf = tfidf.transform(Xtest)
# See output
Xtrain_tfidf.shape

(3000, 61689)

In [ ]:
tfidf.vocabulary_

#### Exercise 1: Find post in the training set that is closest in tf-idf to the first post in the test set (`Xtest[0]`). Print the content of both posts (not the tf-idf vectors).

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances
import numpy as np

first_test_post_vector = Xtest_tfidf[0:1]

# 2. คำนวณ Euclidean distance ระหว่างโพสต์นี้กับโพสต์ทั้งหมดในชุดฝึก
# ผลลัพธ์คือ array ของระยะทาง
distances = euclidean_distances(first_test_post_vector, Xtrain_tfidf)

# 3. ค้นหาดัชนีของโพสต์ที่มีระยะทางน้อยที่สุด
# np.argmin() จะคืนค่าดัชนีของค่าที่น้อยที่สุดใน array
# เนื่องจาก 'distances' เป็นเมทริกซ์ 1xN เราจึงต้องใช้ distances[0]
closest_post_index = np.argmin(distances[0])

# 4. ดึงเนื้อหาของโพสต์มาแสดงผล
closest_post_train_content = Xtrain[closest_post_index]
first_test_post_content = Xtest[0]

## --- การแสดงผลลัพธ์ ---

print("--- โพสต์แรกในชุดทดสอบ (Xtest[0]) ---")
print(first_test_post_content)
print("\n-------------------------------------")

print(f"--- โพสต์ที่ใกล้เคียงที่สุดในชุดฝึก (Index: {closest_post_index}) ---")
print(closest_post_train_content)
print("\n-------------------------------------")

print(f"ระยะทาง Euclidean ที่ใกล้ที่สุด: {distances[0][closest_post_index]:.4f}")

--- โพสต์แรกในชุดทดสอบ (Xtest[0]) ---
From: v064mb9k@ubvmsd.cc.buffalo.edu (NEIL B. GANDLER)
Subject: Need info on 88-89 Bonneville
Organization: University at Buffalo
Lines: 10
News-Software: VAX/VMS VNEWS 1.41
Nntp-Posting-Host: ubvmsd.cc.buffalo.edu


 I am a little confused on all of the models of the 88-89 bonnevilles.
I have heard of the LE SE LSE SSE SSEI. Could someone tell me the
differences are far as features or performance. I am also curious to
know what the book value is for prefereably the 89 model. And how much
less than book value can you usually get them for. In other words how
much are they in demand this time of year. I have heard that the mid-spring
early summer is the best time to buy.

			Neil Gandler


-------------------------------------
--- โพสต์ที่ใกล้เคียงที่สุดในชุดฝึก (Index: 2577) ---
From: v063kcbp@ubvmsd.cc.buffalo.edu (MITCH)
Subject: Thanks! (Backing Up Masters)
Keywords: Misled, Confused, Advice, Comprendo!
Organization: University at Buffalo
Lines: 

### Classify with k-Nearest Neighbor (kNN) ([documentation](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# Define model
nb = KNeighborsClassifier(n_neighbors=3)

# Fit the model
nb.fit(Xtrain_tfidf, ytrain)

KNeighborsClassifier(n_neighbors=3)

Evaluate on the test set using [`classification_report`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)

We will focus on the [F1-score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html)

In [ ]:
from sklearn.metrics import classification_report
import numpy as np

ypred = nb.predict(Xtest_tfidf)

np.array(train.target_names)[ypred]
#report classification scores
print(classification_report(ytest, ypred))

              precision    recall  f1-score   support

           0       0.43      0.62      0.51        21
           1       0.33      0.62      0.43        21
           2       0.38      0.62      0.47        26
           3       0.35      0.47      0.40        34
           4       0.47      0.44      0.45        34
           5       0.70      0.54      0.61        26
           6       0.44      0.55      0.49        22
           7       0.70      0.57      0.63        28
           8       0.75      0.73      0.74        33
           9       0.67      0.64      0.65        25
          10       0.83      0.70      0.76        27
          11       0.80      0.80      0.80        20
          12       0.50      0.21      0.29        24
          13       0.79      0.65      0.71        23
          14       0.82      0.64      0.72        28
          15       0.76      0.66      0.70        29
          16       0.54      0.62      0.58        21
          17       0.82    

### Combine all methods into a [pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html)

In [ ]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([('tfidf',TfidfVectorizer()), ('nb',KNeighborsClassifier())])

# Fit the pipeline to the training set
pipeline.fit(Xtrain, ytrain)

# Make predictions on the test set
ypred = pipeline.predict(Xtest)

# report classification scoresprint
print(classification_report(ytest, ypred))

              precision    recall  f1-score   support

           0       0.18      0.48      0.26        21
           1       0.38      0.48      0.43        21
           2       0.56      0.58      0.57        26
           3       0.52      0.50      0.51        34
           4       0.67      0.53      0.59        34
           5       0.88      0.54      0.67        26
           6       0.50      0.41      0.45        22
           7       0.58      0.54      0.56        28
           8       0.74      0.70      0.72        33
           9       0.82      0.72      0.77        25
          10       0.75      0.56      0.64        27
          11       0.71      0.85      0.77        20
          12       0.31      0.21      0.25        24
          13       0.79      0.48      0.59        23
          14       0.76      0.57      0.65        28
          15       0.67      0.55      0.60        29
          16       0.56      0.67      0.61        21
          17       0.59    

Now we will use [grid search cross-validation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) to find model with the best hyperparameters

![5CV](https://scikit-learn.org/stable/_images/grid_search_cross_validation.png)

In [ ]:
from sklearn.model_selection import GridSearchCV

params = {'tfidf__ngram_range' : [(1, 1),(2, 2)],
                'nb__n_neighbors' : [1, 3, 5 ] }

# Define GridSearchCV
gridcv = GridSearchCV(pipeline, params,
                      scoring='f1_macro', cv=3)

# Fit and cross-validate the model on 3-fold data
gridcv.fit(Xtrain, ytrain)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                       ('nb', KNeighborsClassifier())]),
             param_grid={'nb__n_neighbors': [1, 3, 5],
                         'tfidf__ngram_range': [(1, 1), (2, 2)]},
             scoring='f1_macro')

In [ ]:
gridcv.best_estimator_

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('nb', KNeighborsClassifier(n_neighbors=1))])

In [ ]:
# Make predictions on the test set

ypred = gridcv.predict(Xtest)

# report classification scoresprint
print(classification_report(ytest, ypred))

              precision    recall  f1-score   support

           0       0.29      0.52      0.37        21
           1       0.38      0.43      0.40        21
           2       0.54      0.54      0.54        26
           3       0.46      0.38      0.42        34
           4       0.81      0.50      0.62        34
           5       0.68      0.50      0.58        26
           6       0.41      0.41      0.41        22
           7       0.67      0.57      0.62        28
           8       0.82      0.70      0.75        33
           9       0.69      0.72      0.71        25
          10       0.68      0.56      0.61        27
          11       0.70      0.80      0.74        20
          12       0.43      0.42      0.43        24
          13       0.65      0.57      0.60        23
          14       0.64      0.64      0.64        28
          15       0.48      0.52      0.50        29
          16       0.63      0.57      0.60        21
          17       0.43    

#### Exercise 2:

1. Use grid search 5-fold cross-validation across different values of the following two kNN parameters: `n_neighbors` and `metric`  **on the training set** to find the best model.

2. For the best value of `n_neighbors` and `metric` you found above, compute the `f1_macro` score **on the test set**.
* Print the value of `n_neighbors` and `metric`.
* Print the model's `f1_macro` score.

In [ ]:
params = {'nb__n_neighbors' : [1, 3, 5 ,7] , 'nb__metric' : ['minkowski', 'euclidean', 'manhattan']  }
# Define GridSearchCV
gridcv_test = GridSearchCV(pipeline, params, scoring='f1_macro', cv=5)

# Fit and cross-validate the model on training data
gridcv_test.fit(Xtrain, ytrain)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                       ('nb', KNeighborsClassifier())]),
             param_grid={'nb__metric': ['minkowski', 'euclidean', 'manhattan'],
                         'nb__n_neighbors': [1, 3, 5, 7]},
             scoring='f1_macro')

In [ ]:
# Make predictions on the test set
ypred = gridcv_test.predict(Xtest)

# report classification scoresprint
print(classification_report(ytest, ypred))

              precision    recall  f1-score   support

           0       0.29      0.52      0.37        21
           1       0.38      0.43      0.40        21
           2       0.54      0.54      0.54        26
           3       0.46      0.38      0.42        34
           4       0.81      0.50      0.62        34
           5       0.68      0.50      0.58        26
           6       0.41      0.41      0.41        22
           7       0.67      0.57      0.62        28
           8       0.82      0.70      0.75        33
           9       0.69      0.72      0.71        25
          10       0.68      0.56      0.61        27
          11       0.70      0.80      0.74        20
          12       0.43      0.42      0.43        24
          13       0.65      0.57      0.60        23
          14       0.64      0.64      0.64        28
          15       0.48      0.52      0.50        29
          16       0.63      0.57      0.60        21
          17       0.43    

In [ ]:
print("--- ผลลัพธ์การประเมิน ---")
print(f"ค่า n_neighbors ที่ดีที่สุด: {n_neighbors_best}")
print(f"ค่า metric ที่ดีที่สุด: {metric_best}")
print(f"f1_macro score บนชุดทดสอบ: {f1_macro:.4f}")

--- ผลลัพธ์การประเมิน ---


NameError: name 'n_neighbors_best' is not defined